# Text Classification
* **Created by:** Eric Martinez
* **For:** 3351 - AI-Powered Applications
* **At:** University of Texas Rio-Grande Valley

## Essential Components

What are the main ingredients to solving a problem well with an LLM?

- **Task:** Identify the task you are trying to solve. What are the desired inputs? What are the desired outputs
- **Data:** Identify how those inputs make their way to your system once deployed. Identify where your evaluation data will come from.
- **Metrics:** Identify the key metrics for evaluating the performance of your solution
- **Model:** What model will be used to solve this problem?

## Classification

- this is a common task where you need to apply a label to some text
- "categorizing text into buckets"

## Example Use-Cases
- automated labeling
- sentiment analysis
- automated scoring
- analyzing recording transcripts
- analyzing customer SMS messages
- analyzing incoming chatbot messages
- content filtering / safety

## Common Metrics

How can we _quantify_ the performance of our model?

#### Accuracy

Accuracy is the proportion of correct predictions among the total number of cases processed. 

#### Recall

Recall is the fraction of the positive examples that were correctly labeled by the model as positive

#### Precision

Precision is the fraction of correctly labeled positive examples out of all of the examples that were labeled as positive. 

#### F1 Score

The F1 metric is the harmonic mean of the precision and recall. It can be calculated as: F1 = 2 * (precision * recall) / (precision + recall)

## Applied Example: Spam Classification

We work for a company that is getting an increasingly large amount of spam.

Unfortunately, they cannot afford to pay for a commercial email spam detection tool.

Given the lowering and lowering cost of GPT-3.5-turbo, you suggest building a GPT based spam detection tool.

You found a tiny little dataset that you might be able to use to prototype a solution: https://huggingface.co/datasets/TrainingDataPro/email-spam-classification

#### Essential Components

Let's try to fill this out

- **Task:** 
- **Data:** 
- **Metrics:**
- **Model:**

#### Essential Components

- **Task:** 
    - Inputs: subject - text, body - text
    - Outputs: is_spam - boolean
- **Data:** 
    - Hugging Face Dataset: https://huggingface.co/datasets/TrainingDataPro/email-spam-classification
- **Metrics:**
    - Accuracy
- **Model:**
    - GPT-3.5-turbo

In [ ]:
%pip install -r requirements_exercise_04.txt

#### Visualize our AI model

In [ ]:
from utils.magic import mermaid

In [ ]:
%%mermaid
flowchart LR
subgraph Model: spam_classifier
    direction LR
    prompt_1{"Prompt\n(gpt-3.5-turbo)"}
    transform_1[[ post_process]]
    Input --> |"subject (str)"| prompt_1
    Input --> |"body (str)"| prompt_1
    prompt_1 --> |"is_spam (str)"| transform_1
    transform_1 --> |"is_spam (bool)"| Output
end

#### Spam Classifier - Data

In [ ]:
%pip install datasets

In [ ]:
from datasets import load_dataset

full_dataset = load_dataset("TrainingDataPro/email-spam-classification")

print(full_dataset)

In [ ]:
# Print one sample
print(full_da taset['train'][0])

#### Let's load up a cleaned-up version of the dataset

Each example will have the following format:
    
```
{
    "inputs": ...
    "outputs": ...
}
```

In [ ]:
from utils.example_data import load_email_spam_dataset

In [ ]:
dataset = load_email_spam_dataset()
print(dataset)

#### Spam Classifier - Task
- **Inputs:** subject (str), body (str)
- **Outputs:** is_spam (bool)

In [ ]:
def spam_classifier(subject=None, body=None):  
    outputs = {}

    # obviously, this is not good
    outputs['is_spam'] = False
    
    return outputs

#### Spam Classifier - Metrics
    - Accuracy

In [ ]:
def accuracy_metric(predictions=None, references=None):
    if not predictions:
        raise ValueError("Must supply predictions")
        
    if not references:
        raise ValueError("Must supply references")
        
    if len(predictions)!=len(references):
        raise ValueError("Length of predictions must match number of references")
        
    correct = 0
    total = len(references)
    
    for prediction, reference in zip(predictions, references):
        if prediction == reference:
            correct += 1
            
    score = (correct * 1.0) / total
    
    return score

In [ ]:
# let's test out the accuracy metric
pretend_predictions = [1, 1, 1, 0] 
pretend_references = [1, 1, 0, 0]
accuracy_score = accuracy_metric(predictions=pretend_predictions, references=pretend_references)
print(accuracy_score)

#### Spam Classifier - Evaluation Loop (Hand-rolled)

We need a pipeline for processing all examples and crunching metrics

In [ ]:
# this is a basic implementation of an evaluation loop
predictions = []
references = []

for sample in dataset['train']:
    sample_inputs = sample['inputs']
    sample_outputs = sample['outputs']
    
    prediction = spam_classifier(**sample_inputs)
    
    predictions.append(prediction)
    references.append(sample_outputs)

    
# compute accuracy for `is_spam`
is_spam_predictions = [prediction['is_spam'] for prediction in predictions]
is_spam_references = [reference['is_spam'] for reference in references]
is_spam_accuracy_score = accuracy_metric(predictions=is_spam_predictions, references=is_spam_references)
print(f"`is_spam` accuracy score: {is_spam_accuracy_score}")

#### Spam Classifier - Evaluation Loop (Advanced)

We need to optimize for getting fast feedback and making it easy to experiment:
- all important code in one place, easily understandable
- start on a tiny subset of the data (10 examples)
- be able to visualize where we go wrong
- evaluation-driven development

In [ ]:
from utils.example_data import load_email_spam_dataset
from utils.evaluation import evaluate
from utils.openai import chat_completion

def spam_classifier(subject=None, body=None):  
    outputs = {}
    
    # run through gpt prompt
    # ...
    
    # post-process
    # ...
    
    # set outputs, change this obv
    is_spam = False
    
    outputs['is_spam'] = is_spam
    # return
    return outputs


# Load dataset
dataset = load_email_spam_dataset()

# Define important metrics
metrics = {
    "accuracy": {
        "function": accuracy_metric,
        "only": ["is_spam"]
    }
}

# Perform batch evaluation
results = evaluate(spam_classifier, dataset=dataset, split="train", limit=10, metrics=metrics, debug=True)

# Validate model performance
assert results['is_spam']['accuracy'] >= 0.9, "`is_spam` accuracy must be greater than or equal to 0.9 on the 'train' set"

## Exercise 1: Complete the `spam_classifier` function above using GPT-3.5-turbo.

## Exercise 2: Create a working Gradio interface for using your model

Tips:
- use `gr.Textbox` for text entry
- you can add more lines to a textbox, for example `gr.Textbox(label="Big Box", lines=10)`
- add helpful labels to every input for user-friendliness
- you can make a Textbox non-editable by users by making it non-interactive, check the Gradio docs

In [ ]:
# here is a hello world gradio application
import gradio as gr

def greet(name="Nameless"):
    greeting = f"Hello, {name}"
    return greeting
    

with gr.Blocks() as app:
    name = gr.Textbox(label="Name")
    btn = gr.Button(value="Submit")
    greeting = gr.Textbox(label="Greeting")

    btn.click(
        greet,
        inputs=[name],
        outputs=[greeting],
    )

app.launch()
